# Definição dos Dados

Aqui, serão gerados os dados necessários para a montagem do problema. 


\begin{equation}
    Zi: ataque do inimigo i
\end{equation}

\begin{equation}
    Vi: vida do personagem i
\end{equation}

\begin{equation}
    Yij: resistência do personagem i para o inimigo j
\end{equation}

\begin{equation}
    Dij: distância do personagem i para o inimigo j
\end{equation}

O cálculo da resistência segue a fórmula:
\begin{equation}
    Yij(Vi, Dij)  = (10*Vi)/Dij 
\end{equation}

In [46]:
import random

#Função que gera uma matriz aleatoria dentro do limite, baseado no numero de colunas e linhas
def generate_random_matrix(rows, cols, lim):
    matrix = [[random.randint(lim[0], lim[1]) for _ in range(cols)] for _ in range(rows)]
    return matrix

#Função que gera um vetor aleatorio dentro do limite, baseado no tamanho 
def generate_random_array(length, lim):
    array = [random.randint(lim[0], lim[1]) for _ in range(length)]
    return array

#Função para imprimir a matriz
def print_matrix(matrix):
    for row in matrix:
        print(row)

# Input da quantidade de Personagens e de Mobs
numPerso = 2
numMob = 1

#Limites dos valores de Força, Vida e distancia
limForca = [300, 600]
limVida = [400, 800]
limDist = [5, 300]

# Cria a Matriz de distancia
matrizDist = generate_random_matrix(numPerso, numMob, limDist)

#Cria o Vetor de vida
arrayVida = generate_random_array(numPerso, limVida)

#Cria o Vetor de Força
arrayForca = generate_random_array(numMob, limForca)

#Calcula a Matrix de Resistência baseado na formula
matrizRes = [
        [int(10 * arrayVida[i] / matrizDist[i][j]) for j in range(len(matrizDist[0]))]
        for i in range(len(matrizDist))
    ]

#Imprime os valores gerados
print("Distancia:")
print_matrix(matrizDist)

print("Vida:")
print(arrayVida)

print("Forca:")
print(arrayForca)

print("Resistencia:")
print_matrix(matrizRes)



Distancia:
[55]
[95]
Vida:
[499, 656]
Forca:
[587]
Resistencia:
[90]
[69]


# Cálculo dos parâmetros

Para encontrar a a solução computacional do problema utilizando o Gurobi, é necessário obter todos os parâmetros para se criar o modelo. Esses parâmetros incluem: um vetor dos coeficientes de todas as
variáveis na função objetivo, uma matriz com todos os coeficientes de todas as variáveis em todas as restrições, um vetor com os sinais de todas as restrições (<, =, >), um vetor com os resultados de todas
as restrições, e um vetor de caracteres com os tipos de cada variável.

Na modelagem do nosso problema, temos como variáveis:

\begin{equation}
    x_ij: 1, se o inimigo j ataca o personagem i
\end{equation}

\begin{equation}
       0, caso contrário
\end{equation}

\begin{equation}
    k_ij: distância do personagem i e inimigo j,  se o inimigo j ataca o personagem i
\end{equation}

\begin{equation}
       0, caso contrário
\end{equation}


A função objetivo:

$$maxf = \sum_{j=1}^n \sum_{i=1}^m (z_i - y_ij)x_ij - k_ij$$

E restrições divididas em dois tipos:

* Restrições de Desigualdade

$$\sum_{i=1}^m x_ij + 0k_ij < 1.5, ∀ j\epsilon{1, 2...n}$$

* Restrições de Igualdade

$$d_ij*x_ij - k_ij = 1.5, ∀ j\epsilon{1, 2...n}$$
$$ ∀ i\epsilon{1, 2...m}$$


Por fim, temos todas as variáveis $x_ij$ binárias e todas $k_ij$ contínuas

In [47]:

#coefX são os coeficientes de todas as variáveis de Xij. Aqui é feita a inicialização desta
coefX = [
        [arrayForca[j] - matrizRes[i][j] for j in range(len(matrizRes[0]))]
        for i in range(len(matrizRes))
    ]

#coefK são os coeficientes de todas as variáveis Kij. Aqui é feita a inicialização desta
coefK = [-1] * numMob * numPerso

#Aqui, juntasse os coeficientes de todas as variáveis para criar o vetor de coeficientes na função objetivo
funcObj = []
for i in range(len(coefX)):
    funcObj += coefX[i]
    
funcObj += coefK


#Definição de variáveis para os coeficientes, sinais e resultados das restrições
linhaAtual = 0
itens = 0

sinaisRest = ""
resultRest = []
tiposVar = ""


#coefRestX são os coeficientes de todas as variáveis relacionadas às restrições de desigualdade. Aqui é feita a inicialização desta
coefRestX = [
        [0 for j in range(len(coefX) * len(coefX[0]))]
        for i in range(len(matrizDist[0]))
    ]

linhaPerso = [1]*numPerso

#Nas restrições de desigualdade, cada restrição possui uma sequência de 1 para cada linha de Xij   
for k in range(len(coefRestX)):
    for i in range(len(linhaPerso)):
        coefRestX[k][i + linhaAtual] = linhaPerso[i]  
    linhaAtual += len(linhaPerso)

#Adiciona os 0's referentes as variáveis de Kij
for i in range(len(coefRestX)):
    coefRestX[i] += [0] * len(coefK)

#Atribui os valores de sinal e resultado das restrições em seus vetores
for i in range(len(coefRestX)):
    sinaisRest += '<'
    resultRest += [1.5]
    
#coefRestK são os coeficientes de todas as variáveis relacionadas às restrições de igualdade. Aqui é feita a inicialização desta
coefRestK = [
        [0 for j in range(len(funcObj))]
        for i in range(len(coefX) * len(coefX[0]))
    ]

#Nas restrições de igualdade, os coeficientes que acompanham as variaveis de Xij são iguais a distancia daquele elemento, e os de Kij são -1, para todo i e j 
linhaAtual = 0
for i in range(len(coefRestK)):
    coefRestK[i][i] = matrizDist[linhaAtual][i%len(matrizDist[0])]
    coefRestK[i][i+len(coefRestK)] = -1
    linhaAtual += 1
       
#Atribui os valores de sinal e resultado das restrições em seus vetores
for i in range(len(coefRestK)):
    sinaisRest += '='
    resultRest += [0]
 
#Junta todos os coeficientes das restrições em um vetor   
coefRest = []

for i in range(len(coefRestX)):
    coefRest += coefRestX[i]
    
for i in range(len(coefRestK)):
    coefRest += coefRestK[i]
    

#Preenche o vetor com os tipos de cada variável
for i in range(len(coefX) * len(coefX[0])):
    tiposVar += 'B'
    
for i in range(len(coefK)):
    tiposVar += 'C'

#Impressão dos valores
print(funcObj)
print(coefRest)
print(sinaisRest)
print(resultRest)
print(tiposVar)

[497, 518, -1, -1]
[1, 1, 0, 0, 55, 0, -1, 0, 0, 95, 0, -1]
<==
[1.5, 0, 0]
BBCC


# Criação do Script em R

Com todos os parâmetros gerados, agora é possível criar um script em R que colocará esses parâmetros personalizados dentro do modelo a ser abalisado pelo Gurobi.

O modelo a ser seguido é:

```
model$A <- matrix(c(1, 1, 0, 0, 26, 0, -1, 0, 0, 173, 0, -1), nrow=3, byrow=T)
model$obj <- c(-59, 121, -1, -1)
model$rhs <- c( 1, 0, 0)
model$sense <- c('<', '=', '=')
model$vtype <- c('B', 'B', 'C', 'C')
```

In [48]:
#Cria comando para os coeficientes das restrições
linhaRest = ""
sparse = "model$A <- matrix(c("
for i in range(len(coefRest)):
    linhaRest += str(coefRest[i])
    if(i != len(coefRest) - 1):
        linhaRest += ", "
    
    sparse += linhaRest
    linhaRest = ""
    
sparse += "), nrow="
sparse += str(len(resultRest))
sparse += ", byrow=T)"

#Cria comando para os coeficientes da função objetivo
obj = "model$obj <- c("
for i in range(len(funcObj)):
    obj += str(funcObj[i])
    if(i != len(funcObj) - 1):
        obj += ", "
obj += ")"

#Cria comando para os resultados das restrições
rhs = "model$rhs <- c("
for i in range(len(resultRest)):
    rhs += str(resultRest[i])
    if(i != len(resultRest) - 1):
        rhs += ", "
rhs += ")"

#Cria comando para os sinais das restrições
sense = "model$sense <- c("
for i in range(len(sinaisRest)):
    sense += "'" + str(sinaisRest[i]) + "'"
    if(i != len(sinaisRest) - 1):
        sense += ", "
sense += ")"

#Cria comando para os tipos das variaveis
type = "model$vtype <- c("
for i in range(len(tiposVar)):
    type += "'" + str(tiposVar[i]) + "'"
    if(i != len(tiposVar) - 1):
        type += ", "
type += ")"

#Imprime a parte do código gerada
print(sparse)
print(obj)
print(rhs)
print(sense)
print(type)



model$A <- matrix(c(1, 1, 0, 0, 55, 0, -1, 0, 0, 95, 0, -1), nrow=3, byrow=T)
model$obj <- c(497, 518, -1, -1)
model$rhs <- c(1.5, 0, 0)
model$sense <- c('<', '=', '=')
model$vtype <- c('B', 'B', 'C', 'C')


Agora, com o código em mãos, basta inserir a parte modificável no código no RStudio

```
library("slam")
library("gurobi")

model <- list()
model$modelsense <- 'max'

#Colar a construção do modelo aqui

result <- gurobi(model)
result$x

if (result$status == 'OPTIMAL') {
  print(result$objval)
  print(result$x)
} else {
  cat('Optimization returned status:', formatC(result$status), '\n')
}

```